In [1]:
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, OpticalElement,
                             RevolutionQuadricMirror, PlaneFilm)
from pyoptix.optimize import focus

hc = h*c/eV
pyoptix.output_notebook()

intialzing SR library
OptiX library initialized


Loading BokehJS ...

In [2]:
lambda_laser = 1080e-9 # m
def harmonique(N=21):
    return lambda_laser/N

# Definition des optiques 

In [3]:
helimag = Beamline(name="Helimag")
source = GaussianSource(name="Generation")
m1 = RevolutionQuadricMirror(name="M1")
foc = PlaneFilm(name="Focalisation")
spectro = PlaneFilm(name="Spectrometre")

In [4]:
helimag.chains["Complete"] = [source, m1, foc, spectro]
helimag.active_chain = "Complete"

Chaîne Complete:
	Generation -> M1 -> Focalisation -> Spectrometre 


## Definition des paramètres optiques

In [5]:
source.nrays = 10000
largeur_1_sur_e2 = 100e-6 #100µm
fwhm_source = largeur_1_sur_e2/1.699
sigma_source = fwhm_source/2.35
source.sigma_x = sigma_source 
source.sigma_y = sigma_source 
source.sigma_x_div = 1e-3 # 1 mrad
source.sigma_y_div = 1e-3 # 1 mrad
source.next = m1

In [6]:
p = 1.80 # m
q = 0.06 # m
print("inverse G", p/q)
print("expecting spot size FHWH", 2.35*source.sigma_x*q/p)
m1.inverse_p = -1/p
m1.inverse_q = 1/q
m1.theta0 = 10*degree
m1.theta = 10*degree
m1.phi = 90*degree
m1.next = foc
m1.set_recording("input")

inverse G 30.000000000000004
expecting spot size FHWH 1.9619383951343926e-06


In [7]:
foc.phi = -90*degree
m1.distance_from_previous = p
foc.distance_from_previous = q
spectro.distance_from_previous = 0.5

foc.set_recording("input")
spectro.set_recording("input")

In [8]:
helimag.draw_active_chain()

Generation -> M1 -> Focalisation -> Spectrometre 


In [9]:
helimag.clear_impacts(clear_source=True)
helimag.align(harmonique(21))
helimag.generate(harmonique(21))
helimag.radiate()

1

In [10]:
foc.get_diagram()

,X,Y,dX,dY,Lambda
0,2.153000e-07,1.149943e-06,-0.001027,-0.000688,5.142857e-08
1,4.316846e-08,-2.684115e-07,-0.014641,-0.017067,5.142857e-08
2,5.345940e-07,-1.429033e-08,0.033419,-0.050390,5.142857e-08
3,-1.786880e-07,-6.698165e-07,0.015214,0.060755,5.142857e-08
4,2.143508e-06,1.391002e-06,0.009314,0.018685,5.142857e-08
...,...,...,...,...,...
9995,-1.089832e-06,6.109917e-07,0.014248,0.039728,5.142857e-08
9996,1.058683e-06,1.421328e-07,-0.061284,-0.050520,5.142857e-08
9997,-4.547975e-07,-2.299132e-07,0.001228,-0.037386,5.142857e-08
9998,-1.269012e-06,1.565653e-06,0.025456,-0.007664,5.142857e-08


In [11]:
foc.show_diagram(display="all", orthonorm=False)

Helimag Complete


({'xy': ColumnDataSource(id='1189', ...),
  'xxp': ColumnDataSource(id='1190', ...),
  'yyp': ColumnDataSource(id='1191', ...)},
 [Column(id='1349', ...), Column(id='1507', ...), Column(id='1665', ...)])

In [12]:
from pyoptix.optimize import slider_optimizer
from bokeh.plotting import show
show(slider_optimizer(foc, "distance_from_previous", (0.029, 0.09), .0001, screen=foc,
                 wavelength=harmonique(21), display="yyp", light_spd=True, nrays=1000))

Helimag Complete


In [13]:
erreur_pente = 1e-6 # rad RMS
diag = foc.get_diagram()
estimation_sigma_reel_x = np.sqrt(diag["X"].std()**2+(2*q*erreur_pente)**2)
estimation_fwhm_reel_x = estimation_sigma_reel_x*2.35
print(estimation_sigma_reel_x)
print(estimation_fwhm_reel_x)

8.669457749529213e-07
2.037322571139365e-06
